## GEE with Python

conda create -n gee python=3.8

conda activate gee

pip install earthengine-api geemap windows-curses

conda install jupyter

conda install ipykernel

python -m ipykernel install --user --name gee --display-name "Python (gee)"

jupyter notebook

In [17]:
import ee
import geemap

# Authentification et initialisation de Google Earth Engine
ee.Authenticate()
ee.Initialize()

# Importation des données raster

In [18]:
# Fonction pour obtenir les images raster par année et sélectionner la première bande
def importation_rasters(annee):
    nom_raster = f"202406_Global_Pf_Parasite_Rate_BFA_{annee}"
    asset_id = f'projects/ee-khadijaaatou180702/assets/{nom_raster}'
    return ee.Image(asset_id).select(0)

# Liste des années pour les rasters
annees = list(range(2000, 2023))

# Charger toutes les images raster et sélectionner la première bande
rasters = [importation_rasters(annee) for annee in annees]

# Créer une collection à partir de la liste d'images
stack_raster = ee.ImageCollection(rasters)

Cherchons le raster portant les taux moyens de parasite entre 2000 et 2022 pour chaque pixel

In [19]:
# Calculer la moyenne sur la collection
moyenne_raster = stack_raster.mean()

In [20]:
from ipyleaflet import ImageOverlay
# Paramètres de visualisation
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'green', 'yellow', 'red']
}

# Créer une 
# Créer une instance de carte unique
Map = geemap.Map(center=[0, 0], zoom=2)

# Fonction pour visualiser une image en ajoutant une couche à la carte
def visualize_image(image, vis_params, title):
    Map.addLayer(image, title, vis_params)

# Visualiser chaque statistique
visualize_image(moyenne_raster, 'moyenne_raster' , vis_params)


# Ajouter une barre de couleurs (légende automatique)
Map.add_colorbar(vis_params=vis_params, label="Valeur", layer_name='moyenne_raster')

# Ajouter une flèche du nord
north_arrow_url = 'https://upload.wikimedia.org/wikipedia/commons/e/eb/North_arrow.svg'  # URL de l'image de la flèche du nord
north_arrow = ImageOverlay(
    url=north_arrow_url,
    bounds=[[0, 0], [0.1, 0.1]],  # Ajuster les coordonnées pour placer la flèche correctement
    opacity=0.8
)
Map.add_layer(north_arrow)

#Pour afficher la carte
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Superposons ce raster avec un shapefile 

In [21]:
import geemap
import ee

# Initialiser l'API Earth Engine
ee.Initialize() #Cela permetau script d'accéder à l'ensemble des fonctionnalités et des données disponibles dans la plateforme Earth Engine.

# Paramètres de visualisation du raster
vis_params = {
    'min': 0,
    'max': 1,
    #'palette': ['blue', 'green', 'yellow', 'red']
}


# Charger le shapefile (FeatureCollection)
shapefile = ee.FeatureCollection('projects/ee-khadijaaatou180702/assets/bfa_admbnda_adm3_igb_20200323') 

# Créer une carte
Map = geemap.Map(center=[0, 0], zoom=2)

# Ajouter le raster à la carte
Map.addLayer(moyenne_raster, vis_params, 'Raster')

# Ajouter le shapefile à la carte
Map.addLayer(shapefile, {'color': 'black'}, 'Shapefile')

# Afficher la carte avec les deux couches superposées
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Calculons, à présent, les moyennes des pixels par niveau pour avoir les taux de parasite chez les enfants pour le pays, chaque région, chaque province et chaque département

In [23]:
import geemap
import ee

# Charger les shapefiles des régions, provinces et du pays
pays = ee.FeatureCollection('projects/ee-khadijaaatou180702/assets/bfa_admbnda_adm0_igb_20200323') 
regions = ee.FeatureCollection('projects/ee-khadijaaatou180702/assets/bfa_admbnda_adm1_igb_20200323')  
provinces = ee.FeatureCollection('projects/ee-khadijaaatou180702/assets/bfa_admbnda_adm2_igb_20200323')  
communes = ee.FeatureCollection('projects/ee-khadijaaatou180702/assets/bfa_admbnda_adm3_igb_20200323') 

# Fonction pour calculer les moyennes par zone
def statistiques_par_zone(feature_collection, image, statistic_type='mean'):
    stats = image.reduceRegions(
        collection=feature_collection,
        reducer=ee.Reducer.mean(),
        scale=30
    )
    return stats


# Calcul pour l'ensemble du pays
moyenne_pays = statistiques_par_zone(pays, moyenne_raster)
pays_result = moyenne_pays.getInfo()

# Calcul par région
moyenne_regions = statistiques_par_zone(regions, moyenne_raster)
regions_result = moyenne_regions.getInfo()

# Calcul par province
moyenne_provinces = statistiques_par_zone(provinces, moyenne_raster)
provinces_result = moyenne_provinces.getInfo()

# Calcul par commune
moyenne_commune = statistiques_par_zone(communes, moyenne_raster)
communes_result = moyenne_commune.getInfo()




In [24]:
# Afficher les résultats
print("Moyenne pour le pays :")
for feature in pays_result['features']:
    nom_pays = feature['properties']['ADM0_FR']  
    moyenne_valeur = feature['properties']['mean']
    print(f"Pays: {nom_pays}, Moyenne des valeurs des pixels: {moyenne_valeur}")
    


Moyenne pour le pays :
Pays: Burkina Faso, Moyenne des valeurs des pixels: 0.5515284458355636


In [25]:
print("\nMoyennes par région :")
for feature in regions_result['features']:
    nom_region = feature['properties']['ADM1_FR']  
    moyenne_valeur = feature['properties']['mean']
    print(f"Région: {nom_region}, Moyenne des valeurs des pixels: {moyenne_valeur}")




Moyennes par région :
Région: Boucle du Mouhoun, Moyenne des valeurs des pixels: 0.588347845724836
Région: Cascades, Moyenne des valeurs des pixels: 0.5553158279612954
Région: Centre, Moyenne des valeurs des pixels: 0.38142843214033134
Région: Centre-Est, Moyenne des valeurs des pixels: 0.5146212316149344
Région: Centre-Nord, Moyenne des valeurs des pixels: 0.5563787706642385
Région: Centre-Ouest, Moyenne des valeurs des pixels: 0.5818394973996383
Région: Centre-Sud, Moyenne des valeurs des pixels: 0.500187854412427
Région: Est, Moyenne des valeurs des pixels: 0.5765296859560157
Région: Hauts-Bassins, Moyenne des valeurs des pixels: 0.5333631165982764
Région: Nord, Moyenne des valeurs des pixels: 0.5509315323431959
Région: Plateau-Central, Moyenne des valeurs des pixels: 0.4722754548632032
Région: Sahel, Moyenne des valeurs des pixels: 0.5182482182139102
Région: Sud-Ouest, Moyenne des valeurs des pixels: 0.5952927750503144


In [26]:
print("\nMoyennes par province :")
for feature in provinces_result['features']:
    nom_province = feature['properties']['ADM2_FR'] 
    moyenne_valeur = feature['properties']['mean']
    print(f"Province: {nom_province}, Moyenne des valeurs des pixels: {moyenne_valeur}")



Moyennes par province :
Province: Balé, Moyenne des valeurs des pixels: 0.5857305292623181
Province: Banwa, Moyenne des valeurs des pixels: 0.5727793942701935
Province: Kossi, Moyenne des valeurs des pixels: 0.6044048991698723
Province: Mouhoun, Moyenne des valeurs des pixels: 0.5809540356431531
Province: Nayala, Moyenne des valeurs des pixels: 0.6064716119811142
Province: Sourou, Moyenne des valeurs des pixels: 0.5829449970952151
Province: Comoé, Moyenne des valeurs des pixels: 0.568416093672027
Province: Léraba, Moyenne des valeurs des pixels: 0.4829059361508742
Province: Kadiogo, Moyenne des valeurs des pixels: 0.38142843214033106
Province: Boulgou, Moyenne des valeurs des pixels: 0.47528371543247233
Province: Koulpélogo, Moyenne des valeurs des pixels: 0.5668082222875127
Province: Kourittenga, Moyenne des valeurs des pixels: 0.5047744926539366
Province: Bam, Moyenne des valeurs des pixels: 0.571812221473271
Province: Namentenga, Moyenne des valeurs des pixels: 0.5843352913917237
P

In [27]:

print("\nMoyennes par commune :")
for feature in communes_result['features']:
    nom_commune = feature['properties']['ADM3_FR'] 
    moyenne_valeur = feature['properties']['mean']
    print(f"Communes: {nom_commune}, Moyenne des valeurs des pixels: {moyenne_valeur}")


Moyennes par commune :
Communes: Komki-Ipala, Moyenne des valeurs des pixels: 0.4695694495897671
Communes: Komsilga, Moyenne des valeurs des pixels: 0.41902372195533416
Communes: Koubri, Moyenne des valeurs des pixels: 0.3603503250885049
Communes: Ouagadougou, Moyenne des valeurs des pixels: 0.3116276740232843
Communes: Pabré, Moyenne des valeurs des pixels: 0.4410011382929217
Communes: Saaba, Moyenne des valeurs des pixels: 0.33779571020209465
Communes: Tanguen-Dassouri, Moyenne des valeurs des pixels: 0.43560184886065584
Communes: Bagassi, Moyenne des valeurs des pixels: 0.5793835254413017
Communes: Bana, Moyenne des valeurs des pixels: 0.5717056426803911
Communes: Boromo, Moyenne des valeurs des pixels: 0.5957179424942176
Communes: Fara, Moyenne des valeurs des pixels: 0.5719930741271986
Communes: Ouri, Moyenne des valeurs des pixels: 0.6092189181381377
Communes: Pâ, Moyenne des valeurs des pixels: 0.5712220708134487
Communes: Pompoï, Moyenne des valeurs des pixels: 0.5831608730606

## Visualisons les cartes avec OpenstreetMap

In [28]:
import geemap
import ee

# Initialiser l'API Earth Engine
ee.Initialize()

# Fonction pour mettre à jour le shapefile avec les moyennes calculées
def shp_avec_les_moyennes(feature_collection, image, mean_property_name='Moyenne_Pixels'):
    stats = image.reduceRegions(
        collection=feature_collection,
        reducer=ee.Reducer.mean(),
        scale=30
    )
    # Ajouter les moyennes comme propriété
    return stats.map(lambda feature: feature.set(mean_property_name, feature.get('mean')))

# Fonction pour créer une image à partir des valeurs moyennes
def creer_image(feature_collection, mean_property_name):
    """Crée une image raster à partir des valeurs moyennes des entités."""
    return feature_collection.reduceToImage(
        properties=[mean_property_name],
        reducer=ee.Reducer.first()
    )

# Mettre à jour les shapefiles avec les moyennes
pays_updated = shp_avec_les_moyennes(pays, moyenne_raster, 'Moyenne_Pays')
regions_updated = shp_avec_les_moyennes(regions, moyenne_raster, 'Moyenne_Région')
provinces_updated = shp_avec_les_moyennes(provinces, moyenne_raster, 'Moyenne_Province')
communes_updated = shp_avec_les_moyennes(communes, moyenne_raster, 'Moyenne_Commune')

# Créer des images pour les moyennes de chaque niveau
pays_image = creer_image(pays_updated, 'Moyenne_Pays')
regions_image = creer_image(regions_updated, 'Moyenne_Région')
provinces_image = creer_image(provinces_updated, 'Moyenne_Province')
communes_image = creer_image(communes_updated, 'Moyenne_Commune')

# Calculer les valeurs min et max pour les visualisations dynamiques
min_value = 0
max_value = 1  

# Paramètres de visualisation dynamiques pour les couleurs
vis_params = {
    'min': min_value,
    'max': max_value,
    'palette': ['blue', 'green', 'yellow', 'red']
}

# Créer une carte pour chaque niveau administratif
Map_pays = geemap.Map(center=[12, -1], zoom=6)
Map_regions = geemap.Map(center=[12, -1], zoom=6)
Map_provinces = geemap.Map(center=[12, -1], zoom=6)
Map_communes = geemap.Map(center=[12, -1], zoom=6)

# Ajouter les images à chaque carte avec les paramètres de visualisation
Map_pays.addLayer(pays_image, vis_params, 'Pays - Moyenne')
Map_regions.addLayer(regions_image, vis_params, 'Régions - Moyenne')
Map_provinces.addLayer(provinces_image, vis_params, 'Provinces - Moyenne')
Map_communes.addLayer(communes_image, vis_params, 'Communes - Moyenne')

# Ajouter une légende pour chaque carte
Map_pays.add_colorbar(vis_params=vis_params, label="Moyenne des valeurs des pixels - Pays")
Map_regions.add_colorbar(vis_params=vis_params, label="Moyenne des valeurs des pixels - Régions")
Map_provinces.add_colorbar(vis_params=vis_params, label="Moyenne des valeurs des pixels - Provinces")
Map_communes.add_colorbar(vis_params=vis_params, label="Moyenne des valeurs des pixels - Communes")




Afficher les cartes

In [29]:
print("Carte pour le niveau Pays :")
Map_pays

Carte pour le niveau Pays :


Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [30]:
print("Carte pour le niveau Régions :")
Map_regions

Carte pour le niveau Régions :


Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [31]:
print("Carte pour le niveau Provinces :")
Map_provinces

Carte pour le niveau Provinces :


Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [32]:
Map_communes.addLayer(communes, {'color': 'black'}, 'Shapefile')
print("Carte pour le niveau Communes :")
Map_communes

Carte pour le niveau Communes :


Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

# Essayons now d'afficher les noms des niveaux et les taux moyens correspondants

### Pour le Burkina Faso

In [33]:
# Paramètres de visualisation
vis_params = {
    'min': min_value,
    'max': max_value,
    'palette': ['pink']
}
# Créer la carte pour les pays
Map_pays = geemap.Map(center=[12, -1], zoom=6)
Map_pays.addLayer(pays_image, vis_params, 'Pays - Moyenne')

# Ajouter les labels pour les pays
for feature in pays_updated.getInfo()['features']:
    pays_name = feature['properties']['ADM0_FR']
    moyenne_valeur = feature['properties']['Moyenne_Pays']
    label_text = f"{pays_name}: {moyenne_valeur:.2f}"
    
    # Extraire les coordonnées du centre du pays
    centroid = ee.Feature(feature).geometry().centroid().coordinates().getInfo()
    
    # Ajouter le texte sur la carte
    Map_pays.add_text(
        text=label_text,
        xy=centroid,
        font_size=15,
        font_color='black',
        font_family='monospace'
    )

# Ajouter une légende pour les pays
Map_pays.add_colorbar(vis_params=vis_params, label="Moyenne des valeurs des pixels - Pays")

# Afficher les cartes
print("Carte pour les Régions :")
Map_regions

print("Carte pour les Pays :")
Map_pays

Carte pour les Régions :
Carte pour les Pays :


Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

### Pour les provinces du Burkina Faso

In [ ]:
import geemap
import ee

# Initialiser l'API Earth Engine
ee.Initialize()

# Mettre à jour les shapefiles avec les moyennes calculées
def update_shapefile_with_mean(feature_collection, image, mean_property_name='Moyenne_Pixels'):
    stats = image.reduceRegions(
        collection=feature_collection,
        reducer=ee.Reducer.mean(),
        scale=30
    )
    # Ajouter les moyennes comme propriété
    return stats.map(lambda feature: feature.set(mean_property_name, feature.get('mean')))

# Mettre à jour les entités géographiques avec les moyennes des régions
regions_updated = update_shapefile_with_mean(regions, moyenne_raster, 'Moyenne_Région')

# Créer une image des moyennes des régions
regions_image = regions_updated.reduceToImage(
    properties=['Moyenne_Région'],
    reducer=ee.Reducer.first()
)

# Paramètres de visualisation
vis_params = {
    'min': min_value,
    'max': max_value,
    'palette': ['white', '#fee5d9', '#fcae91', '#fb6a4a', '#de2d26', '#a50f15']
}

# Créer la carte
Map_regions = geemap.Map(center=[12, -1], zoom=6)

# Ajouter l'image des régions à la carte
Map_regions.addLayer(regions_image, vis_params, 'Régions - Moyenne')

# Ajouter les labels avec le texte des noms des régions et les moyennes
for feature in regions_updated.getInfo()['features']:
    region_name = feature['properties']['ADM1_FR']
    moyenne_valeur = feature['properties']['Moyenne_Région']
    label_text = f"{region_name}: {moyenne_valeur:.2f}"
    
    # Extraire les coordonnées du centre de chaque région
    centroid = ee.Feature(feature).geometry().centroid().coordinates().getInfo()
    
    # Ajouter le texte sur la carte
    Map_regions.add_text(
        text=label_text,
        xy=centroid,
        font_size=15,
        font_color='black',
        font_family='monospace'
    )

# Ajouter une légende pour la carte
Map_regions.add_colorbar(vis_params=vis_params, label="Moyenne des valeurs des pixels - Régions")

# Afficher la carte
Map_regions


### Pour les régions du Burkina Faso

In [ ]:
import geemap
import ee

# Initialiser l'API Earth Engine
ee.Initialize()

# Mettre à jour les shapefiles avec les moyennes calculées
def update_shapefile_with_mean(feature_collection, image, mean_property_name='Moyenne_Pixels'):
    stats = image.reduceRegions(
        collection=feature_collection,
        reducer=ee.Reducer.mean(),
        scale=30
    )
    # Ajouter les moyennes comme propriété
    return stats.map(lambda feature: feature.set(mean_property_name, feature.get('mean')))

# Mettre à jour les entités géographiques avec les moyennes pour les provinces
provinces_updated = update_shapefile_with_mean(provinces, moyenne_raster, 'Moyenne_Province')

# Créer une image des moyennes pour les provinces
provinces_image = provinces_updated.reduceToImage(
    properties=['Moyenne_Province'],
    reducer=ee.Reducer.first()
)

# Paramètres de visualisation
vis_params = {
    'min': min_value,
    'max': max_value,
    'palette': ['white', '#fee5d9', '#fcae91', '#fb6a4a', '#de2d26', '#a50f15']
}

# Créer la carte pour les provinces
Map_provinces = geemap.Map(center=[12, -1], zoom=6)
Map_provinces.addLayer(provinces_image, vis_params, 'Provinces - Moyenne')

# Ajouter les labels pour les provinces
for feature in provinces_updated.getInfo()['features']:
    province_name = feature['properties']['ADM2_FR']
    moyenne_valeur = feature['properties']['Moyenne_Province']
    label_text = f"{province_name}: {moyenne_valeur:.2f}"
    
    # Extraire les coordonnées du centre de chaque province
    centroid = ee.Feature(feature).geometry().centroid().coordinates().getInfo()
    
    # Ajouter le texte sur la carte
    Map_provinces.add_text(
        text=label_text,
        xy=centroid,
        font_size=12,
        font_color='black',
        font_family='monospace'
    )

# Ajouter une légende pour les provinces
Map_provinces.add_colorbar(vis_params=vis_params, label="Moyenne des valeurs des pixels - Provinces")

# Afficher la carte pour les provinces
Map_provinces


In [ ]:
import geemap
import ee

# Initialiser l'API Earth Engine
ee.Initialize()

# Mettre à jour les shapefiles avec les moyennes calculées
def update_shapefile_with_mean(feature_collection, image, mean_property_name='Moyenne_Pixels'):
    stats = image.reduceRegions(
        collection=feature_collection,
        reducer=ee.Reducer.mean(),
        scale=30
    )
    # Ajouter les moyennes comme propriété
    return stats.map(lambda feature: feature.set(mean_property_name, feature.get('mean')))

# Mis à jour des entités géographiques avec les moyennes pour les communes
communes_updated = update_shapefile_with_mean(communes, moyenne_raster, 'Moyenne_Commune')

# Vérifier les résultats
communes_info = communes_updated.getInfo()
print("Exemple de données mises à jour pour les communes :")
print(communes_info['features'][0]['properties'])  # Imprimer les propriétés d'une des communes pour vérification

# Créer une image des moyennes pour les communes
communes_image = communes_updated.reduceToImage(
    properties=['Moyenne_Commune'],
    reducer=ee.Reducer.first()
)

# Paramètres de visualisation
vis_params = {
    'min': min_value,
    'max': max_value,
    'palette': ['white', '#fee5d9', '#fcae91', '#fb6a4a', '#de2d26', '#a50f15']
}

# Créer la carte pour les communes
Map_communes = geemap.Map(center=[12, -1], zoom=6)
Map_communes.addLayer(communes_image, vis_params, 'Communes - Moyenne')

# Ajouter les labels pour les communes
for feature in communes_info['features']:
    commune_name = feature['properties']['ADM3_FR']
    moyenne_valeur = feature['properties'].get('Moyenne_Commune', 'Non calculée')  # Utilisez get() pour éviter KeyError
    label_text = f"{commune_name}: {moyenne_valeur:.2f}" if moyenne_valeur != 'Non calculée' else f"{commune_name}: Non calculée"
    
    # Extraire les coordonnées du centre de chaque commune
    centroid = ee.Feature(feature).geometry().centroid().coordinates().getInfo()
    
    # Ajouter le texte sur la carte
    Map_communes.add_text(
        text=label_text,
        xy=centroid,
        font_size=12,
        font_color='black',
        font_family='monospace'
    )

# Ajouter une légende pour les communes
Map_communes.add_colorbar(vis_params=vis_params, label="Moyenne des valeurs des pixels - Communes")

# Afficher la carte pour les communes
Map_communes


In [121]:
# Charger les shapefiles des régions, provinces et du pays
pays = ee.FeatureCollection('projects/ee-khadijaaatou180702/assets/bfa_admbnda_adm0_igb_20200323')  # Remplacez par le chemin de votre shapefile du pays
regions = ee.FeatureCollection('projects/ee-khadijaaatou180702/assets/bfa_admbnda_adm1_igb_20200323')  # Remplacez par le chemin de votre shapefile des régions
provinces = ee.FeatureCollection('projects/ee-khadijaaatou180702/assets/bfa_admbnda_adm2_igb_20200323')  # Remplacez par le chemin de votre shapefile des provinces
communes = ee.FeatureCollection('projects/ee-khadijaaatou180702/assets/bfa_admbnda_adm3_igb_20200323') # Remplacez par le chemin de votre shapefile des communes

# Fonction pour calculer les moyennes par zone
def zonal_statistics(feature_collection, image, scale=30, statistic_type='mean'):
    reducer = None
    # Sélectionner le réducteur en fonction du type de statistique
    if statistic_type == 'mean':
        reducer = ee.Reducer.mean()
    elif statistic_type == 'median':
        reducer = ee.Reducer.median()
    elif statistic_type == 'sum':
        reducer = ee.Reducer.sum()
    elif statistic_type == 'min':
        reducer = ee.Reducer.min()
    elif statistic_type == 'max':
        reducer = ee.Reducer.max()
    else:
        raise ValueError("Type de statistique non supporté : choisissez parmi 'mean', 'median', 'sum', 'min' ou 'max'.")

    # Calculer les statistiques par zone
    stats = image.reduceRegions(
        collection=feature_collection,
        reducer=reducer,
        scale=scale
    )
    return stats

# Exemple de calcul de la moyenne pour l'ensemble du pays
try:
    moyenne_pays = zonal_statistics(pays, moyenne_raster, statistic_type='mean')
    pays_result = moyenne_pays.getInfo()
except Exception as e:
    print(f"Erreur lors du calcul pour le pays : {e}")

# Exemple de calcul de la moyenne par région
try:
    moyenne_regions = zonal_statistics(regions, moyenne_raster, statistic_type='mean')
    regions_result = moyenne_regions.getInfo()
except Exception as e:
    print(f"Erreur lors du calcul pour les régions : {e}")

# Exemple de calcul de la moyenne par province
try:
    moyenne_provinces = zonal_statistics(provinces, moyenne_raster, statistic_type='mean')
    provinces_result = moyenne_provinces.getInfo()
except Exception as e:
    print(f"Erreur lors du calcul pour les provinces : {e}")

# Exemple de calcul de la moyenne par commune
try:
    moyenne_commune = zonal_statistics(communes, moyenne_raster, statistic_type='mean')
    communes_result = moyenne_commune.getInfo()
except Exception as e:
    print(f"Erreur lors du calcul pour les communes : {e}")

# Affichage des résultats avec vérification de la présence des clés
def afficher_resultats(resultat, niveau_administratif):
    if 'features' in resultat:
        print(f"\nMoyennes par {niveau_administratif} :")
        for feature in resultat['features']:
            nom = feature['properties'].get(f'ADM{niveau_administratif}_FR', 'Nom non disponible')
            moyenne_valeur = feature['properties'].get('mean', 'Valeur non disponible')
            print(f"{niveau_administratif}: {nom}, Moyenne des valeurs des pixels: {moyenne_valeur}")
    else:
        print(f"Aucun résultat trouvé pour les {niveau_administratif}.")

# Afficher les moyennes pour chaque niveau administratif
afficher_resultats(pays_result, "pays")
afficher_resultats(regions_result, "région")
afficher_resultats(provinces_result, "province")
afficher_resultats(communes_result, "commune")


KeyboardInterrupt: 